In [2]:
import pandas as pd
import os

diretorio = 'C:/Users/Henri.Souza/Desktop/Henri/VSCODE/projetos/cepas5anos/datas/raw'

csv_arquivos = [f for f in os.listdir(diretorio) if f.endswith('.csv')]

lista_dfs = []

for arquivo in csv_arquivos:
    caminho = os.path.join(diretorio, arquivo)
    
    df = pd.read_csv(
        caminho,
        encoding='latin1',
        sep=';',
        decimal=',',
        skiprows=1  
    )
    
    lista_dfs.append(df)

dados = pd.concat(lista_dfs, ignore_index=True)


Dados Brutos importados

Acesso aos arquivos no repositório do github

In [3]:
dados.isnull().sum()

ANO                      0
MES                      0
SENADOR                  0
TIPO_DESPESA             0
CNPJ_CPF                 0
FORNECEDOR               0
DOCUMENTO             3979
DATA                     0
DETALHAMENTO         36383
VALOR_REEMBOLSADO        0
COD_DOCUMENTO            0
dtype: int64

In [ ]:
dados = dados[~dados['DOCUMENTO'].isnull()]

In [ ]:
dados = dados[~dados['DETALHAMENTO'].isnull()]

In [ ]:
dados.isnull().sum()

ANO                  0
MES                  0
SENADOR              0
TIPO_DESPESA         0
CNPJ_CPF             0
FORNECEDOR           0
DOCUMENTO            0
DATA                 0
DETALHAMENTO         0
VALOR_REEMBOLSADO    0
COD_DOCUMENTO        0
dtype: int64

In [ ]:
dados.shape

(54798, 11)

In [ ]:
dados['VALOR_REEMBOLSADO'] = pd.to_numeric(dados['VALOR_REEMBOLSADO'], errors= 'coerce')

In [ ]:
dados['CNPJ_CPF'] = dados['CNPJ_CPF'].astype(str).replace(r'\D', '', regex = True)

In [ ]:
dados['COMPETENCIA'] = pd.to_datetime(dados['ANO'].astype(str) + '/' + dados['MES'].astype(str), format = '%Y/%m')


In [ ]:
dados['COMPETENCIA'] = dados['COMPETENCIA'].dt.to_period('M')


In [ ]:
dados['TIPO_DESPESA']  = dados['TIPO_DESPESA'].str.strip().str.upper()

In [ ]:
dados['DETALHAMENTO'] = dados['DETALHAMENTO'].str.strip().str.upper()

In [ ]:
dados['COD_DOCUMENTO'] = pd.to_numeric(dados['COD_DOCUMENTO'], errors='coerce')

In [ ]:
dados['VALOR_REEMBOLSADO'] = dados['VALOR_REEMBOLSADO'].astype(str).replace('.',',')

In [ ]:
dados

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2018,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,NaN,02/01/2018,Despesa com pagamento de aluguel de imóvel par...,6000.0,2088462
1,2018,1,ACIR GURGACZ,"Passagens aéreas, aquáticas e terrestres nacio...",63.764.211/0001-10,TRANSCONTINENTAL AGÊNCIA DE VIAGENS LTDA,M2CJMM,02/01/2018,"Companhia Aérea: AVIANCA, Localizador: M2CJMM....",274.99,2088476
2,2018,1,ACIR GURGACZ,"Passagens aéreas, aquáticas e terrestres nacio...",63.764.211/0001-10,TRANSCONTINENTAL AGÊNCIA DE VIAGENS LTDA,SFEXYF,02/01/2018,"Companhia Aérea: LATAM, Localizador: SFEXYF. P...",1094.29,2088528
3,2018,1,ACIR GURGACZ,"Passagens aéreas, aquáticas e terrestres nacio...",63.764.211/0001-10,TRANSCONTINENTAL AGÊNCIA DE VIAGENS LTDA,V64HTR,24/01/2018,"Companhia Aérea: AZUL, Localizador: V64HTR. Pa...",1553.99,2088475
4,2018,2,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,002/18,01/02/2018,Despesa com pagamento de aluguel de imóvel par...,6000.0,2088477
...,...,...,...,...,...,...,...,...,...,...,...
93843,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,WIXHAI,06/12/2022,"Companhia Aérea: LATAM, Localizador: WIXHAI. P...",2893.04,2191398
93844,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,WITOLM,09/12/2022,"Companhia Aérea: GOL, Localizador: WITOLM. Pas...",1180.19,2192272
93845,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,THPKVQ,20/12/2022,"Companhia Aérea: TAM, Localizador: THPKVQ. Pas...",2671.9,2192274
93846,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,QNN9HX,21/12/2022,"Companhia Aérea: AZUL, Localizador: QNN9HX. Pa...",1334.31,2192244
